<h3>准备数据</h3>

<h4>读取数据</h4>

In [2]:
import pandas as pd
from pandas import DataFrame
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, Flatten
from tensorflow.keras.backend import clear_session
from sklearn.metrics import mean_squared_error